In [2]:
import pandas as pd

In [13]:
diagnosis_df = pd.read_csv('../raw/trainingSet/training_SyncDiagnosis.csv')
patient_df = pd.read_csv('../raw/trainingSet/training_SyncPatient.csv')
transcript_df = pd.read_csv('../raw/trainingSet/training_SyncTranscript.csv')
conversion_df = pd.read_csv('../raw/trainingSet/training_SyncTranscriptDiagnosis.csv')
medication_df = pd.read_csv('../raw/trainingSet/training_SyncMedication.csv')

patient_df.head()

,PatientGuid,DMIndicator,Gender,YearOfBirth,State,PracticeGuid
0,FB6EFC3D-1A20-4497-9CBD-00027CC5D220,0,M,1929,SD,7BF4DAD8-5F67-4985-B911-20C9E89A3737
1,C6746626-6783-4650-A58F-00065649139A,0,F,1985,TX,E7101967-2FF1-4B0F-8129-B0B429D1D15C
2,E05C6E8F-779F-4594-A388-000C635AE4D3,0,F,1984,NJ,FC01A799-1CAF-464F-A86F-8A666AB86F32
3,EAEBD216-F847-4355-87B2-000D942E08F0,0,M,1959,OH,EEBC95EF-79BE-4542-892E-98D3166BAB20
4,C7F10A80-4934-42D2-8540-000FBEBA75C8,0,F,1990,FL,677BA32E-B4C4-48F2-86E4-08C42B135401


In [16]:
medication_df.head()

,MedicationGuid,PatientGuid,NdcCode,MedicationName,MedicationStrength,Schedule,DiagnosisGuid,UserGuid
0,0D6B339C-AB1F-4FF1-B231-000026C414C9,A7F1591C-9DB8-4F12-A0B8-15AC12C2B801,23490544301,Digoxin oral tablet,125 mcg (0.125 mg),NaN,DA833B28-CE04-4547-A94F-8A4E6C187A3B,1247EA46-61C3-4E91-ACF1-9C54D920ED05
1,72ADD073-CDA0-4F85-B6CB-00008B645E90,7A88D55F-22A9-4126-861D-92F0DA2ED701,143126730,Lisinopril oral tablet,10 mg,NaN,3ED5B90C-22A1-46EE-9CF5-027E684627D8,89049E64-8E9C-4863-8CF2-B6874235FA14
2,D35E505A-02F8-4EFD-A460-00042F85EF3E,F0B80C93-D371-44BE-B464-530C97FE6C02,456069801,Tessalon (benzonatate) oral capsule,200 mg,NaN,0B6C3285-1D8C-4C0F-9928-95EC226CB2FE,7511BCEC-9E46-4808-99A7-5B30778B6E84
3,FFB036B9-A636-48F2-967C-0005F8D78574,FF32CBDA-DDB1-4730-81C5-ACB0F321C2C5,60432060504,Promethazine VC Plain (phenylephrine-promethaz...,5 mg-6.25 mg/5 mL,NaN,DE2E15A5-5080-4141-A061-C81EC739C0A4,7CA1DC13-E602-4296-805F-A64DC8950C5A
4,AB095685-A703-4D81-8678-0009396BF105,1ECD93C8-75BD-4D3C-8D15-E2A3D0373090,247211730,Lexapro (escitalopram) oral tablet,10 mg,NaN,16B189A4-359C-41E1-BAA3-3E081EA24C41,92EBF1FE-3BE4-4367-83DD-C34F58B0B6C6


In [18]:
def dense_dataframe(df, on):
    """dense the dataframe by combining entries of same value of a given column"""
    lst = []
    for index, value in df.iteritems(): # loop through columns in the dataframe
        # combines rows of values of each column to a list, and store the lists into a list
        lst.append(df.groupby(on)[index].apply(list)) 
        
    processed_df = pd.concat(lst, axis=1) # convert the list of lists of values to a dataframe 
    processed_df[on] = processed_df[on].apply(lambda x:x[0]) # retrieve the same value of 'on' column from the list    
    return processed_df

processed_conversion_df = dense_dataframe(conversion_df, 'TranscriptGuid')
processed_transcripts_df = dense_dataframe(transcript_df, 'PatientGuid')
processed_medications_df = dense_dataframe(medication_df, 'PatientGuid')
processed_transcripts_df = processed_transcripts_df.merge(processed_medications_df, on='PatientGuid')

In [19]:
conversion_dict1 = processed_conversion_df.set_index('TranscriptGuid')['DiagnosisGuid'].to_dict()
conversion_dict2 = diagnosis_df.set_index('DiagnosisGuid').to_dict()

In [20]:
def get_diagnoses(transcripts):
    diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all = [], [], [], [], [], []

    for i in transcripts:
        if i in conversion_dict1:
            diag = conversion_dict1[i]
            diagnoses.append(diag)

            descriptions_one, acute_one, icd9_one, start_one, stop_one = [], [], [], [], []
            for k in diag:
                descriptions_one.append(conversion_dict2['DiagnosisDescription'][k])
                acute_one.append(       conversion_dict2['Acute'][k])
                icd9_one.append(        conversion_dict2['ICD9Code'][k])
                start_one.append(       conversion_dict2['StartYear'][k])
                stop_one.append(        conversion_dict2['StopYear'][k])
            
            descriptions_all.append(descriptions_one)
            acute_all.append(acute_one)
            icd9_all.append(icd9_one)
            start_all.append(start_one)
            stop_all.append(stop_one)
            
    return diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all

def combine_data(conversion, transcripts):
    """ Matches diagnosis data with each transcript """
    
    transcripts['CombinedData'] = transcripts.TranscriptGuid.apply(get_diagnoses)
    
    transcripts['DiagnosisGuid']        = [x[0] for x in transcripts['CombinedData']]
    transcripts['DiagnosisDescription'] = [x[1] for x in transcripts['CombinedData']]
    transcripts['Acute']                = [x[2] for x in transcripts['CombinedData']]
    transcripts['ICD9Code']             = [x[3] for x in transcripts['CombinedData']]
    transcripts['StartYear']            = [x[4] for x in transcripts['CombinedData']]
    transcripts['StopYear']             = [x[5] for x in transcripts['CombinedData']]
        
    return transcripts
        
combined_data = combine_data(processed_conversion_df, processed_transcripts_df)
combined_data.to_csv('combined_data.csv')

In [21]:
combined_data = pd.read_csv('combined_data.csv')
combined_data.head()

,Unnamed: 0,TranscriptGuid,PatientGuid,VisitYear,Height,Weight,BMI,SystolicBP,DiastolicBP,RespiratoryRate,...,MedicationStrength,Schedule,DiagnosisGuid,UserGuid_y,CombinedData,DiagnosisDescription,Acute,ICD9Code,StartYear,StopYear
0,0,"['C024B9A8-C201-4013-9DD4-16877BFCBDCF', 'A04B...",00023761-9D8D-445B-874C-2424CC7CF620,"[2009, 2009, 2009]","[68.0, 67.0, 67.0]","[182.0, 184.0, 185.0]","[27.670000000000002, 28.815000000000001, 28.97...","[122.0, 134.0, 120.0]","[76.0, 86.0, 62.0]","[16.0, nan, nan]",...,['2 mg'],[nan],"[['C3120220-1285-4FE1-A289-5C327786AAFA', '487...",['5A5D58CC-8874-4DCB-8E38-ED2B89FAC111'],"([['C3120220-1285-4FE1-A289-5C327786AAFA', '48...",[['Hypertrophy (Benign) of prostate ...,"[[0, 0], [0, 0, 0], [0, 0]]","[['600.0', '272.2'], ['600.0', '909.5', '272.2...","[[0, 0], [0, 0, 0], [0, 0]]","[[nan, nan], [nan, nan, nan], [nan, nan]]"
1,1,"['CC8E82FF-2588-4D24-878A-05D0A7A82C68', '0BF9...",0005D9BD-0247-4F02-B7EE-7C1B44825FA1,"[2009, 2010, 0, 0, 0, 0, 2009, 2010, 0, 2010, ...","[nan, 65.0, nan, 65.0, 65.0, 65.0, nan, nan, 6...","[0.0, 135.0, 0.0, 137.0, 137.0, 137.0, 0.0, 0....","[0.0, 22.463000000000001, 0.0, 22.795999999999...","[135.0, 130.0, nan, 128.0, 140.0, 133.0, 125.0...","[75.0, 80.0, nan, 60.0, 59.0, 73.0, 78.0, 82.0...","[nan, nan, nan, nan, nan, nan, 15.0, nan, nan,...",...,['40 mg'],[nan],"[['849004E3-E759-4F6E-ABC3-040DBCF510BF', '248...",['490BBDBE-DBA1-4CCC-A18C-51D00FC13837'],"([['849004E3-E759-4F6E-ABC3-040DBCF510BF', '24...","[['Mixed hyperlipidemia', 'Other psoriasis and...","[[0, 0, 0], [0], [0, 0], [0, 0, 0], [0, 0, 0, 0]]","[['272.2', '696.1', '789.06'], ['696.1'], ['69...","[[2009, 0, 2009], [0], [0, 2009], [0, 0, 2009]...","[[nan, nan, nan], [nan], [nan, nan], [nan, nan..."
2,2,"['582D3C53-0B68-4B68-96FA-0D5664D32392', 'B0D0...",000B4862-7CE7-4EC5-8043-A97FCD74BD78,"[2012, 2012, 2011, 2011, 2012, 2012, 2012, 201...","[65.0, nan, nan, nan, nan, nan, nan, nan, nan,...","[100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[16.638999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[100.0, 80.0, 83.0, nan, nan, nan, nan, nan, 1...","[60.0, 60.0, 56.0, nan, nan, nan, nan, nan, 60...","[18.0, 14.0, 14.0, nan, nan, nan, nan, nan, 20...",...,"['10 mg/24 hr', '250 mcg (0.25 mg)']","[nan, nan]","[['0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11', '79A...","['EE1410C4-FF22-4BE3-9585-45E354C90341', 'EE14...","([['0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11', '79...","[['Impacted cerumen', 'Urinary incontinence, u...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0...","[['380.4', '788.30', '781.2', '458.9', '428.0'...","[[0, 2012, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0],...","[[nan, nan, nan, nan, nan, nan, nan], [nan, na..."
3,3,"['DD03810C-ADF4-4B0D-B43D-01F9FCE71F55', '0393...",00110ABC-DAB9-49E3-A1C8-88BBF8D58109,"[2012, 2011, 2012, 2011, 2009, 2010, 2012, 201...","[nan, nan, nan, nan, 71.0, nan, nan, nan, nan,...","[0.0, 0.0, 0.0, 0.0, 215.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 29.983000000000001, 0.0, ...","[nan, nan, nan, nan, 130.0, nan, nan, nan, nan...","[nan, nan, nan, nan, 80.0, nan, nan, nan, nan,...","[nan, nan, nan, nan, 12.0, nan, nan, nan, nan,...",...,"['-', 'Multiple Vitamins', '500 mg', '81 mg']","[nan, nan, nan, nan]","[['FB912E60-3DB4-42D9-A17D-B1E9171C0DB2'], ['D...","['62C7E69B-EB1E-4711-A6D7-7AB644BEB1F9', '62C7...","([['FB912E60-3DB4-42D9-A17D-B1E9171C0DB2'], ['...","[['Sebaceous cyst'], ['Other enthesopathy of a...","[[0], [0], [0], [0, 0, 0], [0], [0], [0], [0],...","[['706.2'], ['726.79'], ['706.2'], ['455', '27...","[[0], [0], [0], [0, 0, 0], [0], [0], [0], [0],...","[[nan], [nan], [nan], [nan, nan, nan], [nan], ..."
4,4,"['FE53B41D-1653-4090-B6D3-5B5E5BEA87FA', '361B...",002667F4-B9A3-4DE2-875F-8034DD637865,"[2010, 2010, 2010, 2010, 2010, 2010]","[59.0, nan, 59.0, 59.0, 59.0, 59.0]","[109.0, 0.0, 96.0, 100.0, 107.0, 99.0]","[22.013000000000002, 0.0, 19.387999999999998, ...","[160.0, nan, 112.0, 130.0, 160.0, 120.0]","[90.0, nan, 80.0, 70.0, 83.0, 79.0]","[20.0, nan, 20.0, 20.0, 18.0, 